In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
from sentence_transformers import util

In [3]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
model = AutoModel.from_pretrained("microsoft/deberta-v2-xxlarge")


Some weights of the model checkpoint at microsoft/deberta-v2-xxlarge were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
model.parameters().is_cuda

AttributeError: 'generator' object has no attribute 'is_cuda'

In [44]:
def get_similarity(func_model, sentence_1, sentence_2, percent=False):
    # compute embeddings
    inputs_1 = tokenizer.encode_plus(sentence_1, return_tensors='pt', padding=True)
    inputs_2 = tokenizer.encode_plus(sentence_2, return_tensors='pt', padding=True)
    
    with torch.no_grad():
        outputs_1 = func_model(**inputs_1).last_hidden_state[:, 0, :]
        outputs_2 = func_model(**inputs_2).last_hidden_state[:, 0, :]
    
    # compare embeddings
    corr = util.pytorch_cos_sim(outputs_1, outputs_2)
        
    if percent:
        # return in percent (value can be from -1 to 1)
        return (corr.numpy()[0][0] + 1) / 2
    return corr.numpy()[0][0]

In [45]:
get_similarity(model, "Python", "C++", percent=True)

0.7175600826740265